In [1]:
import sys
# sys.path = sys.path[:-1]
sys.path

['/home/rasta/fair-neural-compression-eval/src',
 '/home/rasta/.conda/envs/fnc_eval/lib/python39.zip',
 '/home/rasta/.conda/envs/fnc_eval/lib/python3.9',
 '/home/rasta/.conda/envs/fnc_eval/lib/python3.9/lib-dynload',
 '',
 '/home/rasta/.conda/envs/fnc_eval/lib/python3.9/site-packages']

In [4]:
from predict import perform_inference
from datasets.rfw_latent import RFW_raw, RFW_latent, create_dataloaders
from train import train, write_model, save_model

import torch
import torchvision.models as models
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F

import sys
sys.path.append('/home/rasta/lossy-vae')
from lvae import get_model
from lvae.models.qresvae import zoo

In [6]:
DEVICE = 1
device = torch.device(f'cuda:{DEVICE}' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')
EPOCHS = 5
LEARNING_RATE = 0.01
RATIO = 0.8
BATCH_SIZE = 32

device: cuda:1


In [7]:
# load pre-trained qres model
model_name = 'qres17m'
lmb_value = 64
nc_model = get_model(model_name, lmb_value, pretrained=True).to(device) # weights are downloaded automatically

Downloading: "https://huggingface.co/duanzh0/my-model-weights/resolve/main/qres17m/qres17m-lmb64.pt" to /home/rasta/.cache/torch/hub/checkpoints/qres17m-lmb64.pt
100%|██████████| 63.9M/63.9M [00:01<00:00, 58.3MB/s]


In [19]:
RFW_IMAGES_DIR =  "/media/global_data/fair_neural_compression_data/datasets/RFW/data_64"
RFW_LABELS_DIR = "/media/global_data/fair_neural_compression_data/datasets/RFW/clean_metadata/numerical_labels.csv"
image_ds = RFW_raw(RFW_IMAGES_DIR, RFW_LABELS_DIR)
image_dl_train, image_dl_test = create_dataloaders(image_ds, BATCH_SIZE, RATIO)
latent_ds = RFW_latent(RFW_IMAGES_DIR, RFW_LABELS_DIR, nc_model, device)
latent_dl_train, latent_dl_test = create_dataloaders(latent_ds, BATCH_SIZE, RATIO)

In [14]:
def get_latent(img, nc_model, device):
    ps_layer = nn.PixelShuffle(2)
    img = img.to(device)
    # print(img.shape)
    stats_all = nc_model.forward_get_latents(img)
    latents = [stats_all[latent_block_index]['z'] for latent_block_index in range(12)]
    output = torch.cat((F.interpolate(latents[0], 4),latents[1], latents[2]), 1)
    output = torch.cat((F.interpolate(output, 8),latents[3], latents[4],latents[5], latents[6]), 1)
    output = torch.cat((F.interpolate(output, 16),latents[7], latents[8],latents[9], latents[10], latents[11]), 1)
    output = ps_layer(output)
    return output

In [15]:
class MultiHeadResNet(nn.Module):
    def __init__(self, output_dims):
        super(MultiHeadResNet, self).__init__()
        self.dim_reducing_layer = nn.Conv2d(in_channels=19, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.upsampling_layer = nn.Upsample(size=224, mode='nearest')
        self.resnet = models.resnet18(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet = torch.nn.Sequential(*(list(self.resnet.children())[:-1]))
        self.heads = nn.ModuleDict()
        for head, num_classes in output_dims.items():
            self.heads[head] = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.dim_reducing_layer(x)
        x = self.upsampling_layer(x)
        features = self.resnet(x).squeeze()
        outputs = {}
        for head, head_module in self.heads.items():
            output_logits = head_module(features)
            outputs[head] = F.softmax(output_logits, dim=1)
        return outputs

In [16]:
output_dims = {
    'skin_type': 6,
    'eye_type': 2,
    'nose_type': 2,
    'lip_type': 2,
    'hair_type': 4,
    'hair_color': 5
}

model = MultiHeadResNet(output_dims).to(device)

/home/rasta/.conda/envs/fnc_eval/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rasta/.conda/envs/fnc_eval/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
inp, target, _ = next(iter(image_dl_train))
inp.shape

torch.Size([32, 3, 64, 64])

In [32]:
latents = nc_model.forward_get_latents(inp.to(device))
for i, latent in enumerate(latents):
    print(f'latent {i} shape : {latent["z"].shape}')

latent 0 shape : torch.Size([32, 16, 1, 1])
latent 1 shape : torch.Size([32, 8, 4, 4])
latent 2 shape : torch.Size([32, 8, 4, 4])
latent 3 shape : torch.Size([32, 6, 8, 8])
latent 4 shape : torch.Size([32, 6, 8, 8])
latent 5 shape : torch.Size([32, 6, 8, 8])
latent 6 shape : torch.Size([32, 6, 8, 8])
latent 7 shape : torch.Size([32, 4, 16, 16])
latent 8 shape : torch.Size([32, 4, 16, 16])
latent 9 shape : torch.Size([32, 4, 16, 16])
latent 10 shape : torch.Size([32, 4, 16, 16])
latent 11 shape : torch.Size([32, 4, 16, 16])


In [2]:
import torch

tensor1 = torch.randn(9, 4, 4)
tensor2 = torch.randn(6, 8, 8)

# Concatenate along a specified axis
concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)  # Concatenate along the first dimension

print(concatenated_tensor.shape)  # Output: (15, 8, 8)

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 4 but got size 8 for tensor number 1 in the list.

In [35]:
ps_layer = nn.PixelShuffle(2)


stats_all = nc_model.forward_get_latents(inp.to(device))
latents = [stats_all[latent_block_index]['z'] for latent_block_index in range(12)]
latents
# output = torch.cat((F.interpolate(latents[0], 4),latents[1], latents[2]), 1)
# output = torch.cat((F.interpolate(output, 8),latents[3], latents[4],latents[5], latents[6]), 1)
# output = torch.cat((F.interpolate(output, 16),latents[7], latents[8],latents[9], latents[10], latents[11]), 1)
# output = ps_layer(output)

[tensor([[[[-6.5472e-01]],
 
          [[ 8.1053e-02]],
 
          [[-2.8485e+00]],
 
          [[-2.8365e+00]],
 
          [[-3.6527e-01]],
 
          [[ 6.6499e+00]],
 
          [[-4.2389e+00]],
 
          [[-1.3642e+00]],
 
          [[-8.4642e+00]],
 
          [[-5.3953e+00]],
 
          [[ 7.7518e-02]],
 
          [[ 3.9017e-01]],
 
          [[-6.3385e+00]],
 
          [[-1.9969e-01]],
 
          [[ 2.7673e+00]],
 
          [[ 3.5655e+00]]],
 
 
         [[[-5.6903e+00]],
 
          [[ 1.0254e+00]],
 
          [[-2.7368e+00]],
 
          [[ 9.7169e-01]],
 
          [[-4.2527e-01]],
 
          [[ 5.3354e+00]],
 
          [[-3.3914e+00]],
 
          [[-1.2879e+00]],
 
          [[ 4.4448e-02]],
 
          [[ 5.5687e+00]],
 
          [[-4.7615e-02]],
 
          [[-2.6514e+00]],
 
          [[-2.6532e+00]],
 
          [[ 5.0838e-01]],
 
          [[-5.2305e+00]],
 
          [[ 3.9489e+00]]],
 
 
         [[[-2.3850e+00]],
 
          [[-4.2024e+00]],
 
        

In [36]:
F.interpolate(latents[0], 4).shape

torch.Size([32, 16, 4, 4])